Para el laboratorio hizo uso del para entrenar el modelo de **Redes neuronales** y predecir el número de veces que se comparten en redes sociales (popularidad).

El enlace al dataset es [Online News Popularity](https://archive.ics.uci.edu/dataset/332/online+news+popularity).

El archivo `OnlineNewsPopularity.csv` conjunto de datos resume un conjunto heterogéneo de características sobre artículos publicados por Mashable en un período de dos años. El objetivo es predecir el número de veces que se comparten en redes sociales (popularidad).

Importamos la librerias necesarias:

In [1]:
# used for manipulating directory paths
import os

# Scientific and vector computation for python
import numpy as np

# Plotting library
from matplotlib import pyplot

# Optimization module in scipy
from scipy import optimize

# will be used to load MATLAB mat datafile format
from scipy.io import loadmat


import pandas as pd

#esta tabulate nos sirve para hacer tablas
from tabulate import tabulate

# Librerias para graficación (trazado de gráficos)
from matplotlib import pyplot
# tells matplotlib to embed plots within the notebook

#Para separa el 20% y 80%
from sklearn.model_selection import train_test_split
%matplotlib inline

# Red nueronal

In [2]:
dataset = pd.read_csv('OnlineNewsPopularity.csv', delimiter=',')


# dataset = pd.read_csv('OnlineNewsPopularity.csv', delimiter=',')
#para usarlo a travez de google colab
dataset = dataset.sample(n=20000, random_state=42)

dataset

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
32340,http://mashable.com/2014/09/08/safest-cabbies-...,121,12,1015,0.422018,1.0,0.545031,10,6,33,...,0.100000,0.8,-0.160714,-0.500000,-0.071429,0.000000,0.000000,0.500000,0.000000,2900
10480,http://mashable.com/2013/07/25/3d-printed-rifle/,532,9,503,0.569697,1.0,0.737542,9,0,1,...,0.136364,1.0,-0.157500,-0.250000,-0.100000,0.000000,0.000000,0.500000,0.000000,1300
15370,http://mashable.com/2013/10/30/digital-dinosau...,435,9,232,0.646018,1.0,0.748428,12,3,4,...,0.375000,0.5,-0.427500,-1.000000,-0.187500,0.000000,0.000000,0.500000,0.000000,17700
31592,http://mashable.com/2014/08/27/homer-simpson-i...,134,12,171,0.722892,1.0,0.867925,9,5,0,...,0.500000,0.5,-0.216667,-0.250000,-0.166667,0.400000,-0.250000,0.100000,0.250000,1500
198,http://mashable.com/2013/01/10/creepy-robotic-...,728,11,286,0.652632,1.0,0.800000,5,2,0,...,0.100000,0.6,-0.251786,-0.500000,-0.100000,0.200000,-0.100000,0.300000,0.100000,1400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9922,http://mashable.com/2013/07/15/elon-musk-hyper...,542,9,532,0.558577,1.0,0.644118,20,3,15,...,0.100000,1.0,-0.310000,-0.600000,-0.100000,0.000000,0.000000,0.500000,0.000000,4600
20296,http://mashable.com/2014/02/12/derek-jeter-ret...,330,11,1125,0.393178,1.0,0.561129,5,3,4,...,0.100000,1.0,-0.152192,-0.700000,-0.008333,0.050000,0.095833,0.450000,0.095833,812
14010,http://mashable.com/2013/10/03/salt-ship-design/,462,10,850,0.435583,1.0,0.571977,7,6,21,...,0.100000,1.0,-0.178788,-0.291667,-0.100000,0.000000,0.000000,0.500000,0.000000,1400
39479,http://mashable.com/2014/12/23/tablets-messing...,12,8,1250,0.452145,1.0,0.639300,19,0,1,...,0.033333,1.0,-0.289641,-0.700000,-0.100000,0.000000,0.000000,0.500000,0.000000,1000


## Tratamiento de datos

vamos a eliminar las columnas de nuestro dataset que no necesitamos, en este caso la direccion de url:

In [3]:
#dropeamos la columna de url
dataset = dataset.drop(['url'], axis=1)

dataset

,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
32340,121,12,1015,0.422018,1.0,0.545031,10,6,33,1,...,0.100000,0.8,-0.160714,-0.500000,-0.071429,0.000000,0.000000,0.500000,0.000000,2900
10480,532,9,503,0.569697,1.0,0.737542,9,0,1,1,...,0.136364,1.0,-0.157500,-0.250000,-0.100000,0.000000,0.000000,0.500000,0.000000,1300
15370,435,9,232,0.646018,1.0,0.748428,12,3,4,1,...,0.375000,0.5,-0.427500,-1.000000,-0.187500,0.000000,0.000000,0.500000,0.000000,17700
31592,134,12,171,0.722892,1.0,0.867925,9,5,0,1,...,0.500000,0.5,-0.216667,-0.250000,-0.166667,0.400000,-0.250000,0.100000,0.250000,1500
198,728,11,286,0.652632,1.0,0.800000,5,2,0,0,...,0.100000,0.6,-0.251786,-0.500000,-0.100000,0.200000,-0.100000,0.300000,0.100000,1400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9922,542,9,532,0.558577,1.0,0.644118,20,3,15,21,...,0.100000,1.0,-0.310000,-0.600000,-0.100000,0.000000,0.000000,0.500000,0.000000,4600
20296,330,11,1125,0.393178,1.0,0.561129,5,3,4,1,...,0.100000,1.0,-0.152192,-0.700000,-0.008333,0.050000,0.095833,0.450000,0.095833,812
14010,462,10,850,0.435583,1.0,0.571977,7,6,21,0,...,0.100000,1.0,-0.178788,-0.291667,-0.100000,0.000000,0.000000,0.500000,0.000000,1400
39479,12,8,1250,0.452145,1.0,0.639300,19,0,1,0,...,0.033333,1.0,-0.289641,-0.700000,-0.100000,0.000000,0.000000,0.500000,0.000000,1000


## Analisis del dataset

para tener una mejor vista del los tipos de datos del dataset, hacemos un `info()`:

In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20000 entries, 32340 to 10294
Data columns (total 37 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0    timedelta                     20000 non-null  int64  
 1    n_tokens_title                20000 non-null  int64  
 2    n_tokens_content              20000 non-null  int64  
 3    n_unique_tokens               20000 non-null  float64
 4    n_non_stop_words              20000 non-null  float64
 5    n_non_stop_unique_tokens      20000 non-null  float64
 6    num_hrefs                     20000 non-null  int64  
 7    num_self_hrefs                20000 non-null  int64  
 8    num_imgs                      20000 non-null  int64  
 9    num_videos                    20000 non-null  int64  
 10   average_token_length          20000 non-null  float64
 11   num_keywords                  20000 non-null  int64  
 12   self_reference_min_shares     20000 non-null  

podemo notar que todos son de tipo float, por lo cual no nos traera problemas al momento de entrenar el modelo.

## Separacion del 80% de los datos para entrenamiento y 20% para pruebas

Haremos uso de la libreria `sklearn` haciendo uso de su funcion `train_test_split()`, donde recibe como parametros:



In [6]:
#separamos el 20% para test y el 80% para entrenamiento deld dataset
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

# Seleccionamos las columnas para X y la columna 'shares' para y
X_test = test_dataset.drop([' shares'], axis=1).values
y_test = test_dataset[' shares'].values
m_test = len(y_test)

# Seleccionamos las columnas para X y la columna 'shares' para y
X_train = train_dataset.drop([' shares'], axis=1).values
y_train = train_dataset[' shares'].values
m_train = len(y_train)
# Ahora, X y y deberían contener datos numéricos del 80% del dataset



# imprimir todos las X de datos solo 10
print(X_train)
print(y_train)

[[5.77000000e+02 1.10000000e+01 4.76000000e+02 ... 1.36363636e-01
  4.54545450e-02 1.36363636e-01]
 [1.28000000e+02 9.00000000e+00 8.77000000e+02 ... 1.00000000e-01
  5.00000000e-01 1.00000000e-01]
 [6.71000000e+02 6.00000000e+00 1.42200000e+03 ... 5.00000000e-01
  5.00000000e-01 5.00000000e-01]
 ...
 [1.56000000e+02 1.10000000e+01 4.92000000e+02 ... 0.00000000e+00
  5.00000000e-01 0.00000000e+00]
 [4.22000000e+02 1.30000000e+01 9.86000000e+02 ... 0.00000000e+00
  5.00000000e-01 0.00000000e+00]
 [4.12000000e+02 1.00000000e+01 1.94000000e+02 ... 8.00000000e-01
  5.00000000e-01 8.00000000e-01]]
[ 542 1000 3400 ...  950 3600 3100]


In [7]:
print(f"el 80% de datos es:{X_train.shape[0]} y el 20% es: {X_test.shape[0]}")
print( "la totalidad de datos es: ", dataset.shape[0])

el 80% de datos es:16000 y el 20% es: 4000
la totalidad de datos es:  20000


In [8]:
### Usamos el shape para tener una imagen mas clara de las entradas

#Con esta funcion podemos obtener la cantidad de caracteristicas que estamos usando y la cantidad de ejemplos que contiene nuestra matriz.

print("la cantidad de columnas es: ", X_train.shape[1])
print("la cantidad de ejemplos es: ", X_train.shape[0])

la cantidad de columnas es:  36
la cantidad de ejemplos es:  16000


## capas

In [9]:
# En este apartado definiremos la cantidad de capas que tendra el modelo:

# - **Capa de entrada:** Es la cantidad de caracteristicas con las que cuenta el dataset.

# - **Capa oculta:** esto se define haciendo pruebas.

# - **Capa de salida:**como este modelo es de regresion lineal, se debe usar solo una capa de salida.

# Configurando parametros necesario
input_layer_size  = X_train.shape[1]  # Entrada de 36 caracteristicas,
hidden_layer_size = 5   # 5 unidades ocultas en la capa oculta
num_labels = 1          # solo se tiene una salida

print("capa de entrada es: ",input_layer_size)
print("capa de oculta es: ",hidden_layer_size)
print("capa de salida es: ",num_labels)

capa de entrada es:  36
capa de oculta es:  5
capa de salida es:  1


In [10]:
## Inicializacion de Thetas

# Inicializamos los valores de theta1 y theta2 con valores aleatorios para evitar conflictos:
# Los valores aleatorios aseguran que cada neurona pueda aprender patrones distintos, lo cual es crucial para el buen desempeño de la red.
pesos = {}
pesos['Theta1'] = np.random.rand(hidden_layer_size , input_layer_size + 1)
pesos['Theta2'] = np.random.rand(num_labels, hidden_layer_size + 1)

### Guardamos los valores de theta en nuevas variables

`.ravel()` Convertira las matrices Theta1 y Tetha2 en vectores unidimencionales.



In [11]:
# Al inicializar los pesos con valores aleatorios pequeños, se garantiza que las neuronas en una capa oculta comiencen a 
# aprender diferentes funciones. Esto aumenta la capacidad de representación de la red neuronal y permite que aprenda patrones 
# complejos en los datos de entrada. Además, la aleatorización ayuda a evitar que el algoritmo de aprendizaje se atasque en mínimos locales 
# subóptimos durante el entrenamiento.

Theta1, Theta2 = pesos['Theta1'], pesos['Theta2']


print("El tamaño de Theta1 es de: ",Theta1.ravel().shape)
print("El tamaño de Theta2 es de: ",Theta2.ravel().shape)

El tamaño de Theta1 es de:  (185,)
El tamaño de Theta2 es de:  (6,)


In [12]:
## Concadenamos Theta1 y Theta2 para formar un solo array
nn_params = np.concatenate([Theta1.ravel(), Theta2.ravel()])
print(nn_params.shape)

(191,)


In [13]:
###               Funcion de activacion

# Esta función simplemente pasa la entrada directamente a la salida sin aplicar ninguna transformación no lineal.

def linear_activation(z):
    return z

def linear_gradient(z):
    return np.ones_like(z)

## Funcion de costo

In [14]:
print(nn_params.shape)

(191,)


In [15]:
# permite saber si el modelo está haciendo un buen trabajo al comparar las predicciones del modelo con los valores reales que deberían ser.

# la regularización se utiliza en dos lugares principales para ayudar a prevenir el sobreajuste modificando tanto el cálculo del costo como la 
# actualización de los gradientes de los pesos. 

def nnCostFunction(nn_params,
                   input_layer_size,
                   hidden_layer_size,
                   num_labels,
                   X, y, lambda_=0.0):

    # Reshape nn_params para obtener Theta1 y Theta2
    Theta1 = np.reshape(nn_params[:hidden_layer_size * (input_layer_size + 1)],
                        (hidden_layer_size, (input_layer_size + 1)))
    Theta2 = np.reshape(nn_params[(hidden_layer_size * (input_layer_size + 1)):],
                        (num_labels, (hidden_layer_size + 1)))

    m = y.size  # Número de ejemplos de entrenamiento

    J = 0
    Theta1_grad = np.zeros(Theta1.shape)
    Theta2_grad = np.zeros(Theta2.shape)

    # Propagación hacia adelante
    a1 = np.concatenate([np.ones((m, 1)), X], axis=1)
    z2 = a1.dot(Theta1.T)
    a2 = linear_activation(z2)
    a2 = np.concatenate([np.ones((a2.shape[0], 1)), a2], axis=1)
    z3 = a2.dot(Theta2.T)
    a3 = linear_activation(z3)

    # Asegurar que y tenga la forma adecuada
    y = y.reshape(-1, 1)

    #                Función de costo

    #El término de regularización se añade al costo total para penalizar los valores grandes de los pesos. 
    # Esto ayuda a mantener los pesos pequeños, lo que generalmente conduce a un modelo menos complejo y más generalizable.
    reg_term = (lambda_ / (2 * m)) * (np.sum(np.square(Theta1[:, 1:])) + np.sum(np.square(Theta2[:, 1:])))
    J = (1 / (2 * m)) * np.sum(np.square(a3 - y))
    J += reg_term

    # Retropropagación
    delta_3 = a3 - y
    delta_2 = delta_3.dot(Theta2)[:, 1:] * linear_gradient(z2)

    Delta1 = delta_2.T.dot(a1)
    Delta2 = delta_3.T.dot(a2)

    #                    Regularización de gradientes
    # Esto se hace para asegurar que la penalización aplicada a los pesos a través de la función de costo se refleje también en la forma en que
    # los gradientes ajustan estos pesos durante el entrenamiento.
    Theta1_grad = (1 / m) * Delta1
    Theta1_grad[:, 1:] += (lambda_ / m) * Theta1[:, 1:]
    Theta2_grad = (1 / m) * Delta2
    Theta2_grad[:, 1:] += (lambda_ / m) * Theta2[:, 1:]

    # Desenrollar gradientes
    grad = np.concatenate([Theta1_grad.ravel(), Theta2_grad.ravel()])

    return J, grad

In [16]:
## Normalizacion de los datos


#Al visualizar los datos se puede observar que las caracteristicas tienen diferentes magnitudes, por lo cual se debe transformar cada valor 
# en una escala de valores similares.

def  featureNormalize(X):
    X_norm = X.copy()

    #creamos un array de ceros con una longitud igual al número de columnas en el array X. La variable mu y sigma se inicializa como este array de ceros.
    mu = np.zeros(X.shape[1])
    sigma = np.zeros(X.shape[1])

    #Creamos el promedio de cada columna de X
    mu = np.mean(X, axis = 0)
    sigma = np.std(X, axis = 0)

    sigma[sigma == 0] = 1

    #normalizamos los datos con la siguiente formula
    X_norm = (X - mu) / sigma

    return X_norm, mu, sigma

In [17]:
X_norm, mu, sigma = featureNormalize(X_train)

In [18]:
print("Longitud de nn_params:", len(nn_params))
print("Expected size for Theta2:", (hidden_layer_size + 1) * num_labels)

Longitud de nn_params: 191
Expected size for Theta2: 6


## Calculo del costo

In [19]:
lambda_ = 10000
J, _ = nnCostFunction(nn_params, input_layer_size, hidden_layer_size, num_labels, X_norm, y_train, lambda_)
print('Costo en parametros (cargado de ex4weights): %.6f ' % J)
#print('El costo debe esta cercano a               

Costo en parametros (cargado de ex4weights): 96561263.968157 


## Inicializar pesos aleatorios



La matriz de pesos inicializada aleatoriamente se devuelve como salida de la función.

In [20]:
#La función randInitializeWeights que has definido es un método común para inicializar los pesos de una red neuronal 
# de manera aleatoria. Esta inicialización es crucial para asegurar que la red neuronal comience el aprendizaje con una ruptura de simetría, 
# permitiendo que las neuronas aprendan diferentes funciones.

def randInitializeWeights(L_in, L_out, epsilon_init=0.12):

    W = np.zeros((L_out, 1 + L_in))
    W = np.random.rand(L_out, 1 + L_in) * 2 * epsilon_init - epsilon_init

    return W

In [21]:
print('Inicialización de parámetros de redes neuronales...')

initial_Theta1 = randInitializeWeights(input_layer_size, hidden_layer_size)
initial_Theta2 = randInitializeWeights(hidden_layer_size, num_labels)

# Desenrrollr parametros
initial_nn_params = np.concatenate([initial_Theta1.ravel(), initial_Theta2.ravel()], axis=0)

Inicialización de parámetros de redes neuronales...


In [22]:
## Calculo de Thetas

# Después de haber completado la tarea, cambie el maxiter a uno más grande
# Usando un numero maximo de iteraciones
options= {'maxiter': 10000}

# Tratamos con diferentes valores de lambda_
lambda_ = 1

# define una función anónima (lambda) llamada costFunction.
costFunction = lambda p: nnCostFunction(p, input_layer_size,
                                        hidden_layer_size,
                                        num_labels, X_norm, y_train, lambda_)

# Ahora, costFunction es una función que toma solo un argumento.
# (los parámetros de la red neuronal)
res = optimize.minimize(costFunction,
                        initial_nn_params,
                        jac=True,
                        method='TNC',
                        options=options)

# obtenemos la solución de la optimización
nn_params = res.x

# Obtenemos Theta1 y Theta2 de nn_params
Theta1 = np.reshape(nn_params[:hidden_layer_size * (input_layer_size + 1)],
                    (hidden_layer_size, (input_layer_size + 1)))

Theta2 = np.reshape(nn_params[(hidden_layer_size * (input_layer_size + 1)):],
                    (num_labels, (hidden_layer_size + 1)))

C:\Users\Manuel\AppData\Local\Temp\ipykernel_26108\3311480215.py:17: OptimizeWarning: Unknown solver options: maxiter
  res = optimize.minimize(costFunction,


In [23]:
for i in range(len(Theta1)):
    print("Thetha",i,"es: ", Theta1[i])
    print("")

Thetha 0 es:  [56.56180217  1.61536697  2.15792132  4.89687516  6.87190977 -2.59856486
 -5.3729668   6.36281569 -3.84303595  4.60398604  2.13071001 -5.25389563
  4.19884523 11.10623418  4.05777811 10.15919757  0.23605694 -1.23052361
 -7.72623808  8.97728594 -0.08854751  9.05950888 -1.67402096 -0.16934777
  0.36016393 -0.32746151  1.9959487   0.4489593  -0.9099932   1.89657482
 -8.9446574  -0.11210523 -2.2676405   1.05054947  2.08719331 -0.60323009
  4.62120089]

Thetha 1 es:  [14.3010542   0.17304808  0.21304453 -0.3169197  -2.67772884  0.42815104
  0.75293362  2.02755037  0.60096107  2.5079373   1.69394782  0.59528476
  0.93179615  4.37488259  4.51696587  4.90158757 -0.61145538  0.21419963
 -2.86365983  3.87592949 -0.67569465  1.9080532   0.15462704  0.70266865
  1.83219796 -0.97312264 -0.21609617  1.73959228  0.41934263  1.01616504
 -2.55493863 -2.9159619  -2.4875912   2.23049507  0.83217668 -0.60039877
  1.53673315]

Thetha 2 es:  [22.10166466  0.37670532  0.6153139  -0.43380101 -4.

In [24]:
for i in range(len(Theta2)):
    print("Thetha",i,"es: ", Theta1[i])
    print("")

Thetha 0 es:  [56.56180217  1.61536697  2.15792132  4.89687516  6.87190977 -2.59856486
 -5.3729668   6.36281569 -3.84303595  4.60398604  2.13071001 -5.25389563
  4.19884523 11.10623418  4.05777811 10.15919757  0.23605694 -1.23052361
 -7.72623808  8.97728594 -0.08854751  9.05950888 -1.67402096 -0.16934777
  0.36016393 -0.32746151  1.9959487   0.4489593  -0.9099932   1.89657482
 -8.9446574  -0.11210523 -2.2676405   1.05054947  2.08719331 -0.60323009
  4.62120089]



## Funcion para realizar predicciones

Predecir la salida de una entrada dada una red neuronal entrenada, Genera la salida prevista de X dados los pesos entrenados de un neural

In [25]:
def predict(Theta1, Theta2, X):

    # Compute the output using the trained weights
    h1 = np.dot(np.concatenate([np.ones((X.shape[0], 1)), X], axis=1), Theta1.T)
    output = np.dot(np.concatenate([np.ones((h1.shape[0], 1)), h1], axis=1), Theta2.T)

    return output

In [26]:
#Creamos la matriz con 6 ejemplos para hacer la prueba predecir un precio de cada ejemplo:
matriz_datos = np.array([
    [731.0, 9.0, 255.0, 0.604743080614, 0.999999993289, 0.79194630341, 4.9137254902, 4.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.799755687423, 0.0500466753998, 0.0500962518137, 0.0501006734234, 0.0500007119405, 0.341245791246, 0.148947811448, 0.043137254902, 0.0156862745098, 0.733333333333, 0.266666666667, 0.286914600551, 0.0333333333333, 0.7, -0.11875, -0.125, -0.1, 0.0, 0.0, 0.5, 0.0], 
    [731.0, 9.0, 255.0, 0.604743080614, 0.999999993289, 0.79194630341, 4.9137254902, 4.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.799755687423, 0.0500466753998, 0.0500962518137, 0.0501006734234, 0.0500007119405, 0.341245791246, 0.148947811448, 0.043137254902, 0.0156862745098, 0.733333333333, 0.266666666667, 0.286914600551, 0.0333333333333, 0.7, -0.11875, -0.125, -0.1, 0.0, 0.0, 0.5, 0.0], 
    [731.0, 13.0, 1072.0, 0.41564561695, 0.999999998565, 0.540889525766, 4.6828358209, 7.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0286328101715, 0.0287935517322, 0.0285751849112, 0.028571675324, 0.885426777861, 0.513502122877, 0.281003475691, 0.0746268656716, 0.0121268656716, 0.860215053763, 0.139784946237, 0.411127435065, 0.0333333333333, 1.0, -0.220192307692, -0.5, -0.05, 0.454545454545, 0.136363636364, 0.0454545454545, 0.136363636364], 
    [731.0, 13.0, 244.0, 0.559999997511, 0.999999993197, 0.680272104216, 4.42213114754, 4.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.300293733076, 0.0500009756327, 0.0500010257644, 0.0500001760383, 0.549704089489, 0.331639928699, -0.0923054070113, 0.016393442623, 0.0245901639344, 0.4, 0.6, 0.292424242424, 0.136363636364, 0.433333333333, -0.456481481481, -1.0, -0.125, 0.7, -0.4, 0.2, 0.4], 
    [731.0, 11.0, 723.0, 0.490934448409, 0.999999997845, 0.642241377926, 5.22821576763, 6.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.866656685064, 0.0333372075669, 0.0333354827452, 0.0333356642467, 0.0333349603771, 0.375048678466, 0.182696038139, 0.0636237897649, 0.00829875518672, 0.884615384615, 0.115384615385, 0.340572181442, 0.0333333333333, 1.0, -0.213888888889, -0.6, -0.1, 0.5, 0.5, 0.0, 0.5], 
    [731.0, 14.0, 290.0, 0.611510789167, 0.999999994048, 0.76190475737, 4.06896551724, 9.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0222448007009, 0.306856235824, 0.0222310333686, 0.0222241628314, 0.626443767275, 0.571906729634, 0.31812180244, 0.0620689655172, 0.0103448275862, 0.857142857143, 0.142857142857, 0.438908128908, 0.136363636364, 1.0, -0.177777777778, -0.4, -0.00833333333333, 0.0, 0.0, 0.5, 0.0]
])


#Normalizamos todos
matriz_datos= (matriz_datos - mu) / sigma


#Calculamos la Y predicha de los 6 ejemplos de prediccion
pred = predict(Theta1, Theta2, matriz_datos)

print("las predicciones para los 6 ejemplos son:")
i = 0
for i in range(len(matriz_datos)):
    print("Para el ejemplo ",i+1," es:", pred[i])
    print("")





las predicciones para los 6 ejemplos son:
Para el ejemplo  1  es: [3981.90881353]

Para el ejemplo  2  es: [3370.23393951]

Para el ejemplo  3  es: [4844.32952875]

Para el ejemplo  4  es: [4761.31698144]

Para el ejemplo  5  es: [4076.08181451]

Para el ejemplo  6  es: [4227.76716363]



## Validaciones


In [27]:
#Para hacer las validaciones correspondientes, primero se crea la funcion del **Mean squeared error**

#haciendo calculo del error cuadratico medio:
def mean_squared_error(y_pred, y_actual):
    resta = y_pred - y_actual
    err_cuadrado = np.sum(resta ** 2)
    return err_cuadrado / len(y_pred)

In [28]:
# realiza la normalización de los datos de prueba
X_test = (X_test - mu)/ sigma


#Predicción Usando la Red Neuronal
pred = predict(Theta1, Theta2, X_test)

print(pred)

[[2588.94320397]
 [2847.818122  ]
 [2038.10704569]
 ...
 [7269.97203949]
 [2010.42609237]
 [2236.40612303]]


In [29]:
mse = mean_squared_error(pred, y_test)
#calculamos el error cuadratico medio:
print('Error Cuadrático Medio (MSE) = ' + str(mse))
print('Raíz del Error Cuadrático Medio (RMSE) = ' + str(np.sqrt(mse)))

Error Cuadrático Medio (MSE) = 299343564954.9564
Raíz del Error Cuadrático Medio (RMSE) = 547122.9888744911


Concluciones

R- Hecha las predicciones , se puede determinar que el modelo no reliza las predicciones exactas, esto debido a la diferencia entre el valor esperado y el calculado.

